In [1]:
from PyQt6.QtWidgets import QFileDialog
from layout import ApplicationLayout
import pandas, numpy
from core.data import mapping, extract
from itertools import product
from collections import Counter
from core.optimizer import optimize
from core.data.schema import WeightMinMax, DataFrame
from core.data import transform

data = pandas.DataFrame({"weight": 1, "min": pandas.NA, "max": pandas.NA}, dtype="Int64", index=list(mapping.runes.effects.values()))
empty = "Empty"

class Application(ApplicationLayout):
    def __init__(self):
        super().__init__(minw=1400)
        self.json = None
        self.monsters = None
        self.runes = None

        self.weight_min_max.data = data.T
        self.monster_selector.addItem("Monster")
        self.monster_selector.setDisabled(True)
        for sel in self.runesets_selectors:
            sel.addItems([empty, *sorted(mapping.runes.sets.values())])

        self.button.clicked.connect(self.select_data_file)
        # self.enable_optimizer()


    def gatherSets(self):
        return dict(Counter([sel.value for sel in self.runesets_selectors if sel.value != empty]))

    def gatherWeightMinMax(self) -> DataFrame[WeightMinMax]:
        wmm = self.weight_min_max.data.astype("Int64").T
        wmm.weight = wmm.weight.fillna(0)
        return wmm

    def select_data_file(self, *args):
        file_dialog = QFileDialog(self)
        file_dialog.setFileMode(QFileDialog.FileMode.ExistingFile)
        file_dialog.setNameFilter("JSON Files (*.json)")
        file_dialog.setViewMode(QFileDialog.ViewMode.List)
        if file_dialog.exec():
            file_paths = file_dialog.selectedFiles()
            if file_paths:
                file_path = file_paths[0]  # Get the first selected file
                self.json = extract.json(file_path)
                self.enable_optimizer()

    def enable_optimizer(self):
        self.runes = extract.runes(self.json)
        self.monsters = extract.monsters(self.json)
        self.monster_selector.setEnabled(True)
        self.monster_selector.removeItem(0)
        self.monster_selector.addItems(sorted(self.monsters.name.dropna().values))
        self.button.setText("Optimize")
        self.button.clicked.disconnect(self.select_data_file)
        self.button.clicked.connect(self.optimize)

    def optimize(self):
        filtered_runes = self.runes 
        if not self.allow_equipped_checkbox.isChecked():
            filtered_runes = transform.filter_equipped(self.runes, self.monsters, self.monster_selector.value)
        solution = optimize(filtered_runes, self.gatherWeightMinMax(), self.gatherSets())
        if solution is not None:
            self.results.data = transform.named_monster_runes_view(solution, self.monsters).set_index("slot").drop(columns="spd%")
                

app = Application()
app.exec()

In [2]:
app.runes.drop(columns="spd%")

,monster,slot,set,hp,hp%,atk,atk%,def,def%,spd,crt,crd,res,acc
id,,,,,,,,,,,,,,
45968131124,19918998103,1,Blade,0,0,118,13,0,0,11,15,0,8,5
46050920366,19918998103,2,Fatal,332,5,0,47,35,0,5,15,0,0,0
45890876944,19918998103,3,Blade,139,0,0,0,118,8,6,12,18,0,0
45977263962,19918998103,4,Fatal,0,0,18,13,0,5,0,16,80,0,4
45725036318,19918998103,5,Fatal,1800,0,0,7,0,9,6,16,0,0,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45993908594,0,5,Intangible,1800,0,0,6,0,6,0,18,0,8,0
45983328994,0,6,Intangible,0,0,0,47,0,0,5,20,4,5,0
46092618788,0,6,Intangible,0,7,0,11,11,0,0,0,7,8,0


In [3]:
import pulp

In [4]:
pulp.value(pulp.LpVariable("x"))

In [5]:
from core.optimizer.model import Model

In [6]:
m = Model()

In [7]:
m.objective += pulp.LpVariable("x")

TypeError: Expected <class 'pulp.pulp.LpAffineExpression'> got <class 'pulp.pulp.LpVariable'>